In [ ]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import os

Parses and trims HTML content by removing all attributes from HTML tags and removing line break tags from the content

In [ ]:
def parse_and_trim(content, content_type):
    if content_type == 'HTML':
        soup = BeautifulSoup(content, 'lxml')
    else:
        soup = BeautifulSoup(content, 'lxml')
    for tag in soup.recursiveChildGenerator():
        try:
            tag.attrs = None
        except AttributeError:
            pass
    for linebreak in soup.find_all('br'):
        linebreak.extract()
    return soup

Remove_multiple_spaces to replace multiple spaces with a single space in a string, and find_qrt_date to extract and format a quarterly date from text content.

In [ ]:
def remove_multiple_spaces(string):
    pattern = r'\s+'
    replaced_string = re.sub(pattern, ' ', string)
    return replaced_string


def find_qrt_date(content):
    qtr_date = content.find_all(text=re.compile(
        r'for\s+(the\s+)?(fiscal\s+)?year\s+ended\s+|for\s+the\s+quarter\s+ended\s+|for\s+the\s+quarterly\s+period\s+ended\s+', re.IGNORECASE))
    qtr_match = re.search(
        r'([A-Za-z]+)\s+(\d{1,2}),\s+(\d{4})', qtr_date[0].replace('\n', ''))
    if qtr_match is None:
        qtr_match = qtr_match = re.search(
            r'([A-Za-z]+) (\d{1,2}), (\d{4})', qtr_date[1])
    return remove_multiple_spaces(str(qtr_match.group()))

In [ ]:
headers = {
    'User-Agent': 'ARES CAPITAL CORP'
}
filing_links = pd.read_excel(
    "../ARCC__sec_filing_links.xlsx")
filing_links.head()

In [ ]:
# drops all the amendment filing
filing_links = filing_links.drop(filing_links[filing_links['Form description'].str.contains(
    'amendment', case=False)].index).reset_index(drop=True)
filing_links['Reporting date'] = pd.to_datetime(filing_links['Reporting date'])
filing_links = filing_links[filing_links['Reporting date'] >= '2013-03-31']
filing_links.head()

In [ ]:
date_columns = ['Filing date', 'Reporting date']
for col in date_columns:
    filing_links[col] = pd.to_datetime(filing_links[col], format='%Y-%m-%d')
for col in date_columns:
    filing_links[col] = filing_links[col].dt.strftime("%B %d, %Y")
filing_links.head()

In [ ]:
filing_links.info()

In [ ]:
def extract_tables(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None
    all_tags = soup_content.find_all(True)

    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):
        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')

                new_table = pd.read_html(
                    html_table.prettify(), skiprows=0, flavor='bs4')[0]
                new_table = new_table.applymap(lambda x: unicodedata.normalize(
                    'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                new_table = new_table.replace(
                    r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                new_table = new_table.dropna(how='all', axis=0)

                if master_table is None:
                    master_table = new_table
                else:
                    master_table = pd.concat(
                        [master_table, new_table], ignore_index=True)

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    print(master_table.shape)
    return master_table

In [ ]:
def extract_tables_manual(soup_content, qtr_date):
    date_regex_pattern1 = r'([A-Za-z]+\s+\d{1,2},\s+\d{4})'
    date_regex_pattern2 = r'\bAs\s+of\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})\b'
    master_table = None
    for tag in soup_content.find_all(text=re.compile(date_regex_pattern2)):
        date_str = re.search(date_regex_pattern1, tag.text)
        find_next = tag.find_next().text
        next_line = tag.next.text
        if re.search('dollar amounts', find_next) or re.search('dollar amounts', next_line):
            # print(date_str.group(1))
            if date_str is not None:
                date_str = str(date_str.group(1))
                date_str = unicodedata.normalize('NFKD', date_str)
            if qtr_date.replace(',', '').strip().lower() in date_str.replace(',', '').strip().lower():
                html_table = tag.find_next('table')
                while html_table:
                    new_table = pd.read_html(
                        html_table.prettify(), skiprows=0, flavor='bs4')[0]
                    new_table = new_table.applymap(lambda x: unicodedata.normalize(
                        'NFKD', x.strip().strip(u'\u200b').replace('—', '-')) if type(x) == str else x)
                    new_table = new_table.replace(
                        r'^\s*$', np.nan, regex=True).replace(r'^\s*\$\s*$', np.nan, regex=True)
                    new_table = new_table.dropna(how='all', axis=0)

                    if master_table is None:
                        master_table = new_table
                    else:
                        master_table = pd.concat(
                            [master_table, new_table], ignore_index=True)

                    if date_str.replace(',', '').strip().lower() in 'December 31, 2013'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2014'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Food and Beverage', re.IGNORECASE)):
                            break
                    if date_str.replace(',', '').strip().lower() in 'December 31, 2015'.replace(',', '').strip().lower() or date_str.replace(',', '').strip().lower() in 'December 31, 2016'.replace(',', '').strip().lower():
                        if html_table.find(text=re.compile(r'Computers and Electronics', re.IGNORECASE)):
                            break
                    html_table = html_table.find_next('table')

    master_table = master_table.applymap(
        lambda x: x.strip().strip(u'\u200b') if type(x) == str else x)
    master_table = master_table.replace(r'^\s*$', np.nan, regex=True).replace(
        r'^\s*\$\s*$', np.nan, regex=True).replace(r'^\s*\)\s*$', np.nan, regex=True)
    value = master_table.iloc[:, 8].isna() & ~master_table.iloc[:, 9].isna()
    master_table.loc[value, [8, 9]] = master_table.loc[value, [9, 8]].values

    print(master_table.shape)
    return master_table

In [ ]:
def process_table(soi_table_df, append_str):
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    soi_table_df = soi_table_df.dropna(
        how='all', axis=0).reset_index(drop=True)
    # print('1: ' + str(soi_table_df.shape))

    # Separate header and data
    soi_table_header = soi_table_df.iloc[0].dropna(how='any')
    soi_table_data_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])
    # print('2: ' + str(soi_table_data_df.shape))

    # drops all the rows that contains header
    soi_table_data_df = soi_table_data_df[soi_table_data_df[soi_table_data_df.columns[0]]
                                          != soi_table_data_df.columns[0]]

    # print('3: ' + str(soi_table_data_df.shape))

# keeps the Industry row
    soi_table_data_df['get_Industry'] = None

    for index, row in soi_table_data_df.iterrows():
        if row.count() == 1:
            soi_table_data_df.loc[index+1, 'get_Industry'] = row.iloc[0]
            soi_table_data_df = soi_table_data_df.drop(index)

    soi_table_data_df.insert(0, 'Industry', soi_table_data_df['get_Industry'])
    # Drop rows with only two non-null values becuase all the subtotal contain 2 value only
    soi_table_data_df = soi_table_data_df.dropna(thresh=3)
    soi_table_data_df['Industry'].fillna(method='ffill', inplace=True)

    # replace all the - in the data table with 0
    soi_table_data_df = soi_table_data_df.replace('-', 0, regex=False)

    # fix the all the nan value column , Amortized Cost, Fair Value
    columns_to_fill = ['Amortized Cost', 'Fair Value']
    for col in columns_to_fill:
        col_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc(col.replace(' ', ''))
        next_col_index = col_index + 1
        for i in range(len(soi_table_data_df)):
            current_value = soi_table_data_df.iat[i, col_index]
            if pd.isna(current_value) and next_col_index < len(soi_table_data_df.columns):
                next_valid_index = next((j for j, v in enumerate(
                    soi_table_data_df.iloc[i, next_col_index:], start=next_col_index) if pd.notna(v)), None)

                if next_valid_index is not None:
                    next_value = soi_table_data_df.iat[i, next_valid_index]
                    soi_table_data_df.iat[i, col_index] = next_value
                    soi_table_data_df.iat[i, next_valid_index] = pd.NA

    # drops everything after FairValue
    if 'FairValue' in soi_table_data_df.columns.str.replace(' ', ''):
        start_index = soi_table_data_df.columns.str.replace(
            ' ', '').get_loc('FairValue')
        soi_table_data_df = soi_table_data_df.iloc[:, :start_index+1]

    # Drop rows with only two non-null values this one recheacks

    # Forward fill the first two columns
    col_indices = [0, 1, 2]
    soi_table_data_df.iloc[:, col_indices] = soi_table_data_df.iloc[:, col_indices].fillna(
        method='ffill')
    # print('7: ' + str(soi_table_data_df.shape))

    # Drop rows with all missing values
    soi_table_df = soi_table_df.dropna(how='all', axis=1)
    # print('5: ' + str(soi_table_data_df.shape))

    # Drop columns with all missing values
    soi_table_data_df = soi_table_data_df.dropna(how='all', axis=1)
    # print('6: ' + str(soi_table_data_df.shape))

    cols_to_convert = ['Shares/Units', 'Principal',
                       'Amortized Cost', 'Fair Value']
    for col in cols_to_convert:
        if col.replace(' ', '') in soi_table_data_df.columns.str.replace(' ', ''):
            col_index = soi_table_data_df.columns.str.replace(
                ' ', '').get_loc(col.replace(' ', ''))
            converted_data = pd.to_numeric(
                soi_table_data_df.iloc[:, col_index], errors='coerce').fillna(0)
            soi_table_data_df[soi_table_data_df.columns[col_index]
                              ] = converted_data

    soi_table_data_df = soi_table_data_df.reset_index(drop=True)

    # print('Final: ' + str(soi_table_data_df.shape))

    return soi_table_data_df

In [ ]:
count = 0
manual = ['December 31, 2013', 'December 31, 2014',
          'December 31, 2015', 'December 31, 2016']
path = '../ARCC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
for qtr_date, html_link in zip(filing_links['Reporting date'], filing_links['Filings URL']):
    print(qtr_date)
    print('Starting file # ', count)
    response = requests.get(html_link, headers=headers)
    content = parse_and_trim(response.content, 'HTML')
    print('Getting content done for # ', count)
    if qtr_date in manual:
        master_table = extract_tables_manual(content, qtr_date)
    else:
        master_table = extract_tables(content, qtr_date)
    print("Done creating master_table # ", count)
    processed_table_ = process_table(
        master_table, qtr_date.replace(',', ''))
    processed_table_.to_excel(
        writer, sheet_name=qtr_date.replace(',', ''), index=False)
    # processed_table_.to_csv(
        # '../csv_file/'+qtr_date.replace(',', '')+'.csv')
    print("Done processed_table # ", count)
    count += 1
    writer.book.save(path)
writer.close()

# Testing
### To test one file at a time
#### You can use the index of the link to run

In [ ]:

# url = filing_links.iloc[-1]['Filings URL']
# date = filing_links.iloc[-1]['Reporting date']
# url, date
# response = requests.get(url, headers=headers)
# content = parse_and_trim(response.content, 'HTML')
# master_table = extract_tables(content, date)
# process_table_ = process_table(master_table, "")
#process_table_.to_excel("example.xlsx")
#process_table_.to_csv('example.csv')
# process_table_

Second Step

In [ ]:
path = '../ARCC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)

In [ ]:
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
for sheet_name, sheet_df in dataframes.items():
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

In [ ]:
import matplotlib.pyplot as plt

dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df

# Extract DataFrame names and shapes
df_names = []
df_shapes = []
for sheet_name, sheet_df in dataframes.items():
    df_names.append(sheet_name)
    df_shapes.append(sheet_df.shape)

# Create a bar graph
plt.figure(figsize=(10, 6))
plt.barh(df_names, [shape[0] for shape in df_shapes], color='skyblue')
plt.xlabel('Number of Rows')
plt.ylabel('DataFrame Name')
plt.title('DataFrame Shapes')
plt.gca().invert_yaxis()  # Reverse the order of DataFrame names
plt.show()

In [ ]:
for dataframe in dataframes:
    print(dataframes[dataframe].columns.values)

print('Without none columns')
for dataframe in dataframes:
    # Get the columns with "Unnamed" in their names
    unnamed_columns = [
        col for col in dataframes[dataframe].columns if 'Unnamed' in col]
    # Drop the unnamed columns
    dataframes[dataframe].drop(columns=unnamed_columns, inplace=True)
    # Print the updated columns
    print(dataframes[dataframe].columns.values)

In [ ]:
column_pattern = {
    r'^Company\s*\((\d+)\)$': 'Company',
    r'^Interest\(\d+\)\(\d+\)$': 'Interest',
    r'^Acquisition\s+Date$': 'Acquisition Date',
    r'^Amortized\s+Cost$': 'Amortized Cost',
    r'^Interest\s+\(\d+\)\(\d+\)$': 'Interest',
    r'^Fair\s+Value$': 'Fair Value',
    r'^Coupon\s+\(\d+\)$': 'Coupon',
    r'^Reference\s+\(\d+\)$': 'Reference',
    r'^Spread\s+\(\d+\)$': 'Spread',
    r'^Maturity\s+Date$': 'Maturity Date',
    r'^Shares/Units$': 'Shares/Units',
    r'^Principal$': 'Principal',
}

def rename_columns_with_pattern(df):
    df.columns = df.columns.to_series().replace(column_pattern, regex=True)


# Iterate through dataframes and apply the column renaming function
for dataframe in dataframes:
    rename_columns_with_pattern(dataframes[dataframe])

# Print the updated columns for each dataframe
for dataframe in dataframes:
    print(dataframes[dataframe].columns.values)

In [ ]:
os.makedirs("../clean_csv_file", exist_ok=True)

excel_file_name = '../Clean_ARCC_Investment.xlsx'

# Create an Excel writer object
excel_writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')

# Iterate through dataframes and write them to the Excel file
for idx, dataframe in enumerate(dataframes):
    # Write each dataframe to a separate sheet (sheet names will be Sheet1, Sheet2, etc.)
    dataframes[dataframe].to_excel(
        excel_writer, sheet_name=f'{dataframe}', index=False)
    dataframes[dataframe].to_csv('../clean_csv_file/'+dataframe+'.csv')

# Save the Excel file
excel_writer.book.save(excel_file_name)

print(f'Excel file "{excel_file_name}" has been created.')



# ALL DONE